```json
            { 
                "tag": "q_0000_0001", 
                "type": "question", 
                "description": { 
                    "number": "", 
                    "question": "", 
                    "options": [], 
                    "options": null, 
                    "answer": "", 
                    "explanation": "" 
                    }, 
                "caption": [ 
                    "" 
                    ], 
                "file_path": null, 
                "bbox": null 
            }, 
{ 
    "tag": "note_0000_0001", 
    "type": "footnote", 
    "description": "1 이러한 계약을 법적으로는 금전소비대차계약이라고 한다. 차용증서를 영어로는 I owe you.의 소리를 따라 IOU 라고 한다.", 
    "caption": null, 
    "file_path": null, 
    "bbox": null 
}, 
{ 
    "tag": "tb_0000_0001", 
    "type": "table", 
    "description": "", 
    "caption": [], 
    "file_path": "??????/crop/tb_0000_0001.png", 
    "bbox": null 
}, 
{ 
    "tag": "img_0000_0001", 
    "type": "image", 
    "description": null, 
    "caption": [], 
    "file_path": "?????/crop/img_0000_0001.png", 
    "bbox": null 
}
```

In [ ]:
# 원천페이지수
import tools.data_processing.epubstats as ep

ep.check_pdf_pages(4, 'pdf_pages.xlsx')

### Lv2 파일명 변경, Lv3/4/5 폴더 분류

In [1]:
from tools.core.utils import FileManager

i = 1
file_manager = FileManager()
excel = file_manager.load_excel_metadata(i)

In [2]:
excel

,ISBN,도서명,출판일,코퍼스 1분류,코퍼스 2분류,비고,분류
관리번호,,,,,,,
SS0319,9791163478782,금융의 현재와 미래,2020-01-03,금융,기초,시사,Lv3/4
SS0320,9788984059900,요즘 금리 쉬운 경제,2020-04-27,금융,기초,시사,Lv3/4
SS0321,9788960542570,알기 쉬운 보험 세테크 100% 활용법,2020-06-23,보험,기초,세무,Lv3/4
SS0322,9788957614358,공정거래 용어사전,2020-08-10,금융,기초,용어,Lv2
SS0323,9791190820660,1페이지 보험 정리의 기술,2020-08-28,보험,기초,시사,Lv3/4
...,...,...,...,...,...,...,...
SS0420,9791138370714,2025 시대에듀 차량손해사정사 2차 한권으로 끝내기,2025-04-15,보험,수험서,차량손해사정사,Lv3/4
SS0421,9791138392440,新2025 시대에듀 hoa 재경관리사 전과목 핵심이론+적중문제+기출 동형문제 한권으...,2025-04-10,금융,수험서,재경관리사,Lv5
SS0422,9791138392938,2025~2026 시대에듀 PASSCODE 증권투자권유자문인력 실제유형 모의고사 4...,2025-05-10,증권,수험서,투자권유자문인력,Lv5


In [6]:
from tqdm import tqdm
import os

original_path = f"/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C/Lv1"

for isbn in tqdm(os.listdir(original_path)):
    if not isbn.startswith('9'):
        continue
    isbn = int(os.path.splitext(isbn)[0])
    # break
    if isbn in excel.ISBN.to_list():
        os.system(f"cd {original_path} && cp {isbn}.pdf {excel[excel['ISBN'] == isbn].index[0]}.pdf")
        # print(f"cp {original_path}/{isbn}.pdf {excel[excel['ISBN'] == isbn].index[0]}.pdf")
    else:
        print(f"{isbn} not in excel")


 75%|███████▌  | 160/212 [00:02<00:00, 68.95it/s]

9788960542297 not in excel


100%|██████████| 212/212 [00:02<00:00, 73.42it/s] 


In [ ]:
isbn

In [ ]:
import shutil
from tqdm import tqdm
import os, json

# zip_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/zip/250826_1차_원천코퍼스_보완사항반영"
original_path = f"/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C"
final_path = f"/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/{i}C"

error_list = []
for id in tqdm(os.listdir(zip_path)):
# if True:
   # id = 'SS0174'
  if not id.startswith('SS'):
    continue

  id = id.replace('.zip', '')
   
  try:
      lv = excel.loc[id, '분류']
  except Exception as e:
      error_list.append({'id':id, 'error': "excel error"})
      continue
  
  if '/' in lv: 
      lv = lv.replace("/", '_')

  if 'Lv' not in lv:
      continue
  
  # os.system(f"cp {zip_path}/{id}.zip {original_path}/{lv}/{id}.zip && cd {original_path}/{lv} && mkdir {id} && unzip {id}.zip -d {id}")
  # os.system(f"mkdir {final_path}/{lv}/{id} && cp {original_path}/{lv}/{id}/{id}.json {final_path}/{lv}/{id}/ && cp -r {original_path}/{lv}/{id}/crop {final_path}/{lv}/{id}/")
                # && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")
  # 위에 실패했을 때 따로
  os.system(f"cp {original_path}/{lv}/{id}/{id}.json {final_path}/{lv}/{id}/ && cp -r {original_path}/{lv}/{id}/crop {final_path}/{lv}/{id}/")

  try:
      file_path = f"{final_path}/{lv}/{id}/{id}.json"
      with open(file_path, 'r', encoding='utf-8') as f:
          content = json.load(f)
        
      with open(file_path.replace(".json", ".json.bak"), 'w', encoding='utf-8') as f:
          json.dump(content, f, ensure_ascii=False, indent=4)

      updated_content = {
            'file_id': id,
            'ISBN': str(excel.loc[id, 'ISBN']),
            'title': excel.loc[id, '도서명'],
            'cat1_domain': excel.loc[id, '코퍼스 1분류'],
            'cat2_sub': excel.loc[id, '코퍼스 2분류'],
            'cat3_specific': excel.loc[id, '비고'],
            'pub_date': str(excel.loc[id, '출판일'])[:10],
            # 'contents': [],
            'contents': content['contents']
        }
      # break
      with open(file_path, 'w', encoding='utf-8') as f:
               json.dump(updated_content, f, ensure_ascii=False, indent=4)
  
  except Exception as e:
      error_list.append({'id':id, 'error': e})

In [ ]:
error_list

In [ ]:
lv, id

### 업로드

In [ ]:
import os

from_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/evaluation/eval_data/8_multiple_exam_+"
to_path = "yjmoon@oci004:/home/yjmoon/"
# to_path = "yjmoon@oci004:/srv4-data/vanilla_models/"


# # 업로드
os.system(f"scp -r {from_path} {to_path}")
# # for ss in file:
#     # os.system(f"scp -r /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/3C/Lv3_4/{ss} yjmoon@oci004:/home/yjmoon/FINAL/{ss}")

In [ ]:
import os
os.system("scp -r yjmoon@oci004:/home/yjmoon/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/evaluation/eval_data/8_multiple_exam_+/exam_+_result/* /Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/8_multiple_exam_+/exam_+_result")

## Lv2 pdf_page 만들기

In [ ]:
# import pandas as pd
# import os

# base_path = "/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar"
# analysis = {1:'1차 분석', 2:'2차 분석', 3: '3차 분석'}
# buy = {1:'1차 구매', 2:'2차 구매', 3: '3차 구매'}
# i = 3

# excel_analy = pd.read_excel(os.path.join(base_path, 'book_list_ALL.xlsx'), sheet_name=analysis[i], header=3, engine='openpyxl')[['관리번호', 'ISBN', '도서명', '분류']]

In [ ]:
# lv = excel_analy[excel_analy['관리번호'] == tn]['분류'].tolist()[0]
# lv
# # print(lv == 'Lv2')

In [ ]:
# # Fixed version - using Python file operations instead of sed
# import shutil
# from tqdm import tqdm
# import fitz

# dpi = 300
# mat = fitz.Matrix(dpi / 150, dpi / 150)

# # for tn, isbn, lv in tqdm(zip(excel_analy['관리번호'], excel_analy['ISBN'], excel_analy['분류'])):
# for tn in tqdm(excel_analy['관리번호']):
#    # if '/' in lv: 
#    #    lv = lv.replace("/", '_')
   
#    # if 'Lv' not in lv:
#    #    continue
#    lv = excel_analy[excel_analy['관리번호'] == tn]['분류'].tolist()[0]
#    # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mv {isbn}.json {tn}.json")
#    if lv == 'Lv2':
#       os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C/Lv2 && mkdir {tn} && mv {tn}*.json {tn}/ && cd {tn} && mkdir pdf_page")
#       # pdf_page 만들기
#       pdf_path = f"/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C/Lv1/{tn}.pdf"
#       doc = fitz.open(pdf_path)
#       for p, page in enumerate(doc):
#          img = page.get_pixmap(matrix=mat)
#          img.save(f"/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C/Lv2/{tn}/pdf_page/{tn}_{p+1:04d}.png")
      
#       break

#    # SS0041 - SS0045 사이에서 MuPDF error: syntax error: invalid key in dict

## 빈 chapter

In [ ]:
import json
import os

# SS01002.json 파일 경로
file_path = "/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/data/FINAL/2C/Lv3_4/SS01005_workbook/SS01005.json"

# JSON 파일 읽기
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 백업 파일 생성
backup_path = file_path.replace('.json', '.json.bak')
with open(backup_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

# 빈 chapter를 이전 값으로 채우기
last_chapter = None
filled_count = 0

for content in data.get('contents', []):
    current_chapter = content.get('chapter', '')
    
    # chapter가 비어있고 이전 chapter가 있으면 채우기
    if not current_chapter and last_chapter:
        content['chapter'] = last_chapter
        filled_count += 1
    # chapter가 비어있지 않으면 last_chapter 업데이트
    elif current_chapter:
        last_chapter = current_chapter

# 수정된 데이터 저장
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"완료! {filled_count}개의 빈 chapter 필드를 채웠습니다.")
print(f"백업 파일: {backup_path}")


In [ ]:
!python tools/data_processing/epubstats.py --cycle 4 --output_file pdf_pages.xlsx